# Meilisearch | Meilisearch> [Meilisearch](https://meilisearch.com)はオープンソースで、非常に高速、そして極めて関連性の高い検索エンジンです。開発者が迅速な検索体験を構築できるように、優れたデフォルト設定が用意されています。>> > [Meilisearch](https://meilisearch.com) is an open-source, lightning-fast, and hyper relevant search engine. It comes with great defaults to help developers build snappy search experiences.>> Meilisearchを[自己ホストする](https://www.meilisearch.com/docs/learn/getting_started/installation#local-installation)ことも、[Meilisearch Cloud](https://www.meilisearch.com/pricing)上で実行することもできます。>> > You can [self-host Meilisearch](https://www.meilisearch.com/docs/learn/getting_started/installation#local-installation) or run on [Meilisearch Cloud](https://www.meilisearch.com/pricing).Meilisearch v1.3はベクター検索に対応しています。このページでは、Meilisearchをベクターストアとして統合する方法と、それを使用してベクター検索を行う方法について案内します。> Meilisearch v1.3 supports vector search. This page guides you through integrating Meilisearch as a vector store and using it to perform vector search.

## Setup | セットアップ### Launching a Meilisearch instance | Meilisearchインスタンスの起動ベクターストアとして使用するためには、稼働中のMeilisearchインスタンスが必要です。[ローカルでMeilisearchを実行する](https://www.meilisearch.com/docs/learn/getting_started/installation#local-installation)か、[Meilisearch Cloud](https://cloud.meilisearch.com/)のアカウントを作成できます。> You will need a running Meilisearch instance to use as your vector store. You can run [Meilisearch in local](https://www.meilisearch.com/docs/learn/getting_started/installation#local-installation) or create a [Meilisearch Cloud](https://cloud.meilisearch.com/) account.Meilisearch v1.3から、ベクターストレージは実験的な機能です。Meilisearchインスタンスを立ち上げた後、**ベクターストレージを有効にする**必要があります。自己ホスト型のMeilisearchについては、[実験的機能を有効にする](https://www.meilisearch.com/docs/learn/experimental/overview)に関するドキュメントを参照してください。**Meilisearch Cloud**では、プロジェクトの*設定*ページから*ベクターストア*を有効にできます。> As of Meilisearch v1.3, vector storage is an experimental feature. After launching your Meilisearch instance, you need to **enable vector storage**. For self-hosted Meilisearch, read the docs on [enabling experimental features](https://www.meilisearch.com/docs/learn/experimental/overview). On **Meilisearch Cloud**, enable *Vector Store* via your project *Settings* page.ベクターストレージが有効になっているMeilisearchインスタンスが稼働しているはずです。🎉> You should now have a running Meilisearch instance with vector storage enabled. 🎉### Credentials | 資格情報Meilisearchインスタンスとやり取りするために、Meilisearch SDKはホスト（インスタンスのURL）とAPIキーが必要です。> To interact with your Meilisearch instance, the Meilisearch SDK needs a host (URL of your instance) and an API key.**ホスト**> **Host*** **ローカル**では、デフォルトのホストは `localhost:7700` です  > In **local**, the default host is `localhost:7700`* **Meilisearch Cloud**で、プロジェクトの*設定*ページにあるホストを見つけてください  > On **Meilisearch Cloud**, find the host in your project *Settings* page**APIキー**> **API keys**Meilisearchインスタンスは、初期設定で3つのAPIキーを提供します：> Meilisearch instance provides you with three API keys out of the box:* `MASTER KEY` — これはMeilisearchインスタンスを作成する際にのみ使用すべきです  > A `MASTER KEY` — it should only be used to create your Meilisearch instance* `ADMIN KEY` — これはサーバーサイドでのみ使用し、データベースおよびその設定を更新する際に使うものです  > A `ADMIN KEY` — use it only server-side to update your database and its settings* `SEARCH KEY` — フロントエンドアプリケーションで安全に共有できるキー  > A `SEARCH KEY` — a key that you can safely share in front-end applications必要に応じて[追加のAPIキー](https://www.meilisearch.com/docs/learn/security/master_api_keys)を作成することができます。> You can create [additional API keys](https://www.meilisearch.com/docs/learn/security/master_api_keys) as needed.

### Installing dependencies | 依存関係のインストールこのガイドでは、[Meilisearch Python SDK](https://github.com/meilisearch/meilisearch-python)を使用しています。以下のコマンドを実行してインストールできます：> This guide uses the [Meilisearch Python SDK](https://github.com/meilisearch/meilisearch-python). You can install it by running:

In [ ]:
!pip install meilisearch

詳細については、[Meilisearch Python SDKのドキュメント](https://meilisearch.github.io/meilisearch-python/)を参照してください。> For more information, refer to the [Meilisearch Python SDK documentation](https://meilisearch.github.io/meilisearch-python/).

## Examples | 例Meilisearchベクターストアを初期化する方法はいくつかあります：Meilisearchクライアントを提供するか、必要に応じて*URL*と*APIキー*を用意します。私たちの例では、認証情報は環境変数から読み込まれます。> There are multiple ways to initialize the Meilisearch vector store: providing a Meilisearch client or the *URL* and *API key* as needed. In our examples, the credentials will be loaded from the environment.`os`と`getpass`を使用することで、環境変数をノートブック環境で利用可能にすることができます。この手法は、以下に示すすべての例で使用できます。> You can make environment variables available in your Notebook environment by using `os` and `getpass`. You can use this technique for all the following examples.

In [ ]:
import getpass
import os

os.environ["MEILI_HTTP_ADDR"] = getpass.getpass("Meilisearch HTTP address and port:")
os.environ["MEILI_MASTER_KEY"] = getpass.getpass("Meilisearch API Key:")

OpenAIEmbeddingsを使用するためには、OpenAI APIキーを取得する必要があります。> We want to use OpenAIEmbeddings so we have to get the OpenAI API Key.

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

### Adding text and embeddings | テキストと埋め込みを追加するこの例では、Meilisearchベクターストアを初期化することなく、Meilisearchベクターデータベースにテキストを追加する方法を示しています。> This example adds text to the Meilisearch vector database without having to initialize a Meilisearch vector store.

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Meilisearch

embeddings = OpenAIEmbeddings()

In [ ]:
with open("../../modules/state_of_the_union.txt") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

In [ ]:
# Use Meilisearch vector store to store texts & associated embeddings as vector
vector_store = Meilisearch.from_texts(texts=texts, embedding=embeddings)

内部では、Meilisearchはテキストを複数のベクトルに変換します。これにより、以下の例と同様の結果が得られます。> Behind the scenes, Meilisearch will convert the text to multiple vectors. This will bring us to the same result as the following example.

### Adding documents and embeddings | ドキュメントと埋め込みを追加するこの例では、Langchain TextSplitterを使用してテキストを複数のドキュメントに分割し、それからこれらのドキュメントとそれらの埋め込みを一緒に保存します。> In this example, we'll use Langchain TextSplitter to split the text in multiple documents. Then, we'll store these documents along with their embeddings.

In [ ]:
from langchain.document_loaders import TextLoader

# Load text
loader = TextLoader("../../modules/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# Create documents
docs = text_splitter.split_documents(documents)

# Import documents & embeddings in the vector store
vector_store = Meilisearch.from_documents(documents=documents, embedding=embeddings)

# Search in our vector store
query = "What did the president say about Ketanji Brown Jackson"
docs = vector_store.similarity_search(query)
print(docs[0].page_content)

## Add documents by creating a Meilisearch Vectorstore | Meilisearch Vectorstoreを作成してドキュメントを追加します

このアプローチでは、ベクターストアオブジェクトを作成し、そこにドキュメントを追加します。> In this approach, we create a vector store object and add documents to it.

In [ ]:
import meilisearch
from langchain.vectorstores import Meilisearch

client = meilisearch.Client(url="http://127.0.0.1:7700", api_key="***")
vector_store = Meilisearch(
    embedding=embeddings, client=client, index_name="langchain_demo", text_key="text"
)
vector_store.add_documents(documents)

## Similarity Search with score | スコア付きの類似性検索この特定のメソッドを使用すると、クエリに対するドキュメントとその距離スコアを返すことができます。> This specific method allows you to return the documents and the distance score of the query to them.

In [ ]:
docs_and_scores = vector_store.similarity_search_with_score(query)
docs_and_scores[0]

## Similarity Search by vector | ベクトルによる類似性検索

In [ ]:
embedding_vector = embeddings.embed_query(query)
docs_and_scores = vector_store.similarity_search_by_vector(embedding_vector)
docs_and_scores[0]

## Additional resources | 追加のリソースドキュメンテーション> Documentation* [Meilisearch](https://www.meilisearch.com/docs/)  > [Meilisearch](https://www.meilisearch.com/docs/)* [Meilisearch Python SDK](https://python-sdk.meilisearch.com)  > [Meilisearch Python SDK](https://python-sdk.meilisearch.com)オープンソースのリポジトリ> Open-source repositories* [Meilisearchのリポジトリ](https://github.com/meilisearch/meilisearch)  > [Meilisearch repository](https://github.com/meilisearch/meilisearch)* [Meilisearch Python SDK](https://github.com/meilisearch/meilisearch-python)  > [Meilisearch Python SDK](https://github.com/meilisearch/meilisearch-python)